In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
pd.options.display.max_columns = None

import pypyodbc as pyodbc
pyodbc.lowercase = False
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [2]:
#import data source from SQL Server
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=Server_name;"
            "Database=Database_name;"
            "UID=User_name;"
            'PWD=Password;')
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()

data_result = pd.read_sql("""
                        SELECT FormulaName,ProductName,[Quota(SmallestUnit)],Price, QuotationID,NewTenderPackage AS TenderPackage,ResultDate,ValidTo, TenderYear
                        FROM TDI_TenderResult
                        WHERE [Include/Exclude] IS NULL""",
                        cnxn)
data_inviting = pd.read_sql("""
                        SELECT FormulaName,[Quota(SmallestUnit)], QuotationID,NewTenderPackage AS TenderPackage,ResultDate,ValidTo, TenderYear
                        FROM TDI_TenderInviting
                        WHERE [Include/Exclude] IS NULL""",
                        cnxn)

#import data source from excel files
#data_result = pd.read_excel(your_file_location, sheet_name=your_sheet_name)
#data_inviting = pd.read_excel(your_file_location, sheet_name=your_sheet_name)

In [4]:
#copy the data source to prevent time-consuming for importing all the dataset again
df_result = data_result[['FormulaName','ProductName','Quota(SmallestUnit)','Price','QuotationID','TenderPackage','ResultDate','ValidTo','TenderYear']].copy()
df_inviting = data_inviting[['FormulaName','Quota(SmallestUnit)','QuotationID','TenderPackage','ResultDate','ValidTo','TenderYear']].copy()

In [5]:
#INVITING

#df_inviting['ResultDate'] = pd.TimedeltaIndex(df_inviting['ResultDate'], unit='d') + datetime(1900,1,1)
#df_inviting['ValidTo'] = pd.TimedeltaIndex(df_inviting['ValidTo'], unit='d') + datetime(1900,1,1)
df_inviting['ResultDate'] = pd.to_datetime(df_inviting['ResultDate'].str.strip())
df_inviting['ValidTo'] = pd.to_datetime(df_inviting['ValidTo'].str.strip())

df_inviting = df_inviting[~((df_inviting['ValidTo'] < pd.to_datetime('2021-01-01')) | (df_inviting['ResultDate'] > pd.to_datetime('2023-12-31')))]

for i in ['FormulaName','TenderPackage']:
    df_inviting[i] = df_inviting[i].str.upper()

df_inviting['Quota(SmallestUnit)'] = pd.to_numeric(df_inviting['Quota(SmallestUnit)'],errors='coerce')

df_inviting['TenderPackage'][~df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])] = 'Other'
df_inviting['TenderPackage'] = np.where(
    ~df_inviting['TenderPackage'].isin(['GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5','BRANDED']),
    'Other',
    df_inviting['TenderPackage']
)

#RESULT
#df_result['ResultDate'] = pd.TimedeltaIndex(df_result['ResultDate'], unit='d') + datetime(1900,1,1)
#df_result['ValidTo'] = pd.TimedeltaIndex(df_result['ValidTo'], unit='d') + datetime(1900,1,1)
df_result['ResultDate'] = pd.to_datetime(df_result['ResultDate'].str.strip())
df_result['ValidTo'] = pd.to_datetime(df_result['ValidTo'].str.strip())

df_result = df_result[~((df_result['ValidTo'] < pd.to_datetime('2021-01-01')) | (df_result['ResultDate'] > pd.to_datetime('2023-12-31')))]

for i in ['FormulaName','ProductName','TenderPackage']:
    df_result[i] = df_result[i].str.upper()

df_result['Quota(SmallestUnit)'] = pd.to_numeric(df_result['Quota(SmallestUnit)'],errors='coerce')
df_result['Price'] = pd.to_numeric(df_result['Price'],errors='coerce')

df_result['TenderPackage'][~df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])] = 'Other'
df_result['TenderPackageGroup'] = np.where(
    df_result.TenderPackage.isin(['GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5']),
    'GENERIC',
    df_result['TenderPackage']
)

C:\Users\dung.hoang1.nguyen\AppData\Local\Temp\ipykernel_12384\1675901426.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inviting['TenderPackage'][~df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])] = 'Other'
C:\Users\dung.hoang1.nguyen\AppData\Local\Temp\ipykernel_12384\1675901426.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['TenderPackage'][~df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])] = 'Other'


In [6]:
#change the element of "startdates", "enddates", "phasings_quota", "phasings_value" in accordance with your request

df_inviting['avg_quota'] = df_inviting['Quota(SmallestUnit)']/(df_inviting['ValidTo']-df_inviting['ResultDate']+timedelta(days=1)).dt.days
df_inviting['avg_quota'][df_inviting['avg_quota']<0] = 0
df_result['avg_quota'] = df_result['Quota(SmallestUnit)']/(df_result['ValidTo']-df_result['ResultDate']+timedelta(days=1)).dt.days
df_result['avg_quota'][df_result['avg_quota']<0] = 0

startdates = ['2021-01-01','2022-01-01','2023-01-01']
enddates = ['2021-12-31','2022-12-31','2023-12-31']

starts = startdates
ends = enddates

phasings_quota = ['2021_volume','2022_volume','2023_volume']
phasings_value = ['2021_value','2022_value','2023_value']

#INVITING
for start, end, s, e in zip(starts, ends, startdates, enddates):
    df_inviting[start] = s
    df_inviting[end] = e

#transform those just created columns into datetime column because we assign them with text value in the first place
for d in startdates+enddates:
    df_inviting[d] = pd.to_datetime(df_inviting[d])

for start, end, quota, value in zip(starts,ends,phasings_quota,phasings_value):
    df_inviting[quota] = np.where(
        (df_inviting['ResultDate']<=df_inviting[start]) & (df_inviting['ValidTo']>=df_inviting[end]),
            df_inviting['avg_quota']*(df_inviting[end]-df_inviting[start]+timedelta(days=1)).dt.days,
        np.where((df_inviting['ResultDate']<=df_inviting[start])
                & ((df_inviting[start]<=df_inviting['ValidTo']) & (df_inviting['ValidTo']<=df_inviting[end])),
                    df_inviting['avg_quota']*(df_inviting['ValidTo']-df_inviting[start]+timedelta(days=1)).dt.days,
        np.where(((df_inviting[start]<=df_inviting['ResultDate']) & (df_inviting['ResultDate']<=df_inviting[end]))
                & (df_inviting['ValidTo']>=df_inviting[end]),
                    df_inviting['avg_quota']*(df_inviting[end]-df_inviting['ResultDate']+timedelta(days=1)).dt.days,
                    0
    )))

#RESULT
for start, end, s, e in zip(starts, ends, startdates, enddates):
    df_result[start] = s
    df_result[end] = e

#transform those just created columns into datetime column because we assign them with text value in the first place
for d in startdates+enddates:
    df_result[d] = pd.to_datetime(df_result[d])

for start, end, quota, value in zip(starts,ends,phasings_quota,phasings_value):
    df_result[quota] = np.where(
        (df_result['ResultDate']<=df_result[start]) & (df_result['ValidTo']>=df_result[end]),
            df_result['avg_quota']*(df_result[end]-df_result[start]+timedelta(days=1)).dt.days,
        np.where((df_result['ResultDate']<=df_result[start])
                & ((df_result[start]<=df_result['ValidTo']) & (df_result['ValidTo']<=df_result[end])),
                    df_result['avg_quota']*(df_result['ValidTo']-df_result[start]+timedelta(days=1)).dt.days,
        np.where(((df_result[start]<=df_result['ResultDate']) & (df_result['ResultDate']<=df_result[end]))
                & (df_result['ValidTo']>=df_result[end]),
                    df_result['avg_quota']*(df_result[end]-df_result['ResultDate']+timedelta(days=1)).dt.days,
                    0
    )))
    df_result[value] = df_result[quota] * df_result.Price

C:\Users\dung.hoang1.nguyen\AppData\Local\Temp\ipykernel_12384\3743538039.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inviting['avg_quota'][df_inviting['avg_quota']<0] = 0
C:\Users\dung.hoang1.nguyen\AppData\Local\Temp\ipykernel_12384\3743538039.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['avg_quota'][df_result['avg_quota']<0] = 0


In [7]:
#generate a pivot table to map Patent and Off-patent formula based on drug's brand name
pivot_result = pd.pivot_table(df_result[(df_result.TenderPackage != 'Other')],
                values='ProductName', 
                index=['FormulaName'], 
                columns='TenderPackageGroup', 
                aggfunc=pd.Series.nunique,
                margins=True
                )

pivot_result.fillna(0, inplace=True)
pivot_result = pivot_result.reset_index(drop=False)

pivot_result.drop(index=pivot_result.index[-1],axis=0,inplace=True)

pivot_result

TenderPackageGroup,FormulaName,BRANDED,GENERIC,GENERIC 1,All
0,,166.0,1060.0,321.0,1479.0
1,\nCETIRIZIN 10MG TAB,0.0,1.0,0.0,1.0
2,ISOTRETINOIN+ERYTHROMYCIN 0.5MG+20MG EXT,0.0,1.0,0.0,1.0
3,L-ORNITHIN L-ASPARTAT 3G ORAL,0.0,1.0,0.0,1.0
4,0 0INJ,0.0,1.0,0.0,1.0
...,...,...,...,...,...
6805,ZOLEDRONIC ACID 5MG INJ,2.0,6.0,3.0,7.0
6806,ZOLEDRONIC ACID 5MG/100ML INJ,1.0,2.0,1.0,3.0
6807,ZOPICLON 7.5MG TAB,0.0,4.0,0.0,4.0
6808,ZOPICLONE 7.5MG TAB,0.0,5.0,3.0,6.0


In [8]:
#map Patent/Off-patent formula based on belowed code
pivot_result['Group_OPO'] = None
pivot_result['Group_OPO'] = np.where(
    (pivot_result.All > 0) & (pivot_result.BRANDED == pivot_result.All),
    'Patent',
    np.where(
        (pivot_result.BRANDED > 0) & (pivot_result['GENERIC 1'] > 0) & (pivot_result.Group_OPO.isna()),
        'OPO',
        'No Group'
    )
)

In [9]:
#map Patent/Off-patent formula back to tender result and tender inviting (original datasets)
df_result = pd.merge(df_result,pivot_result[['FormulaName','Group_OPO']],on='FormulaName',how='left')
df_inviting = pd.merge(df_inviting,pivot_result[['FormulaName','Group_OPO']],on='FormulaName',how='left')

All market: Volume

In [10]:
#generate function to create report that show amount and contribution
def report(df, values, index):
    report_amount = pd.pivot_table(df,
                   values=values,
                   index=[index],
                   aggfunc='sum',
                   margins=True
                   )
    
    report_contr = pd.pivot_table(df,
                   values=values,
                   index=[index],
                   aggfunc='sum'
                   ).apply(lambda x: x*100/sum(x))

    report = pd.concat([report_amount,report_contr], axis=1).style.format("{:,.2f}")
    
    return report

**1. Value (Result)**

1.1. All market

In [11]:
report(df_result[df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])],
       phasings_value,
       'TenderPackage'
       )

,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value
TenderPackage,,,,,,,,,,,,,,
BRANDED,0.00,"63,175,489,060.20","748,231,326,012.34","3,287,051,160,249.34","8,942,875,963,768.95","8,595,924,164,405.36","14,702,433,439,030.68",nan,35.09,15.46,14.18,17.55,17.23,23.28
GENERIC 1,0.00,"58,111,485,178.22","1,350,610,459,094.19","7,295,947,733,771.94","17,535,963,741,711.07","16,983,844,506,309.10","21,148,028,749,301.98",nan,32.27,27.90,31.47,34.40,34.04,33.49
GENERIC 2,0.00,"17,870,886,548.22","627,708,525,632.65","3,076,890,289,512.45","7,105,420,281,022.75","7,805,038,451,675.53","9,698,695,697,692.51",nan,9.93,12.97,13.27,13.94,15.64,15.36
GENERIC 3,0.00,"11,507,828,476.87","344,621,548,873.65","1,325,712,837,013.05","2,527,343,509,537.19","2,651,969,278,183.01","2,352,559,976,814.60",nan,6.39,7.12,5.72,4.96,5.32,3.73
GENERIC 4,0.00,"20,559,911,745.96","1,412,941,593,092.49","6,215,417,308,203.50","11,904,314,122,649.66","10,731,693,825,742.26","11,724,731,938,195.12",nan,11.42,29.19,26.81,23.36,21.51,18.57
GENERIC 5,0.00,"8,831,065,697.11","357,022,373,018.10","1,980,436,199,952.94","2,955,106,388,305.57","3,121,397,131,378.78","3,523,763,998,113.31",nan,4.90,7.37,8.54,5.80,6.26,5.58
All,0.00,"180,056,666,706.59","4,841,135,825,723.42","23,181,455,528,703.20","50,971,024,006,995.19","49,889,867,357,694.05","63,150,213,799,148.18",nan,nan,nan,nan,nan,nan,nan


1.2. P & OPO

In [12]:
report(df_result[
        (df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])) &
        (df_result.Group_OPO.isin(['Patent','OPO']))
        ],
       phasings_value,
       'TenderPackage'
       )

,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value
TenderPackage,,,,,,,,,,,,,,
BRANDED,0.00,"63,168,829,674.03","746,689,389,153.97","3,276,597,820,428.80","8,936,896,759,105.91","8,594,648,427,195.64","14,690,039,818,838.38",nan,45.65,33.36,33.08,35.70,33.96,41.27
GENERIC 1,0.00,"35,451,315,810.13","651,966,756,112.45","3,041,254,106,418.92","7,820,693,230,749.93","7,992,260,339,111.85","10,179,859,830,986.82",nan,25.62,29.13,30.71,31.24,31.58,28.60
GENERIC 2,0.00,"14,919,197,867.92","202,275,706,741.03","952,356,410,130.60","2,407,062,074,413.83","2,803,921,293,985.00","3,899,575,812,027.38",nan,10.78,9.04,9.62,9.62,11.08,10.96
GENERIC 3,0.00,"6,383,143,124.12","159,114,485,017.35","592,616,631,520.39","1,123,619,657,426.09","1,089,738,888,967.82","1,145,433,588,057.46",nan,4.61,7.11,5.98,4.49,4.31,3.22
GENERIC 4,0.00,"12,651,812,956.14","352,312,929,616.51","1,427,866,490,459.48","3,612,304,832,796.63","3,592,134,254,624.49","4,146,097,481,306.64",nan,9.14,15.74,14.42,14.43,14.19,11.65
GENERIC 5,0.00,"5,805,472,066.21","125,862,221,890.86","613,720,081,357.69","1,131,956,741,053.85","1,237,719,263,384.32","1,529,984,948,393.16",nan,4.20,5.62,6.20,4.52,4.89,4.30
All,0.00,"138,379,771,498.54","2,238,221,488,532.16","9,904,411,540,315.88","25,032,533,295,546.24","25,310,422,467,269.12","35,590,991,479,609.85",nan,nan,nan,nan,nan,nan,nan


1.3. Patent

In [13]:
report(df_result[
        (df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])) &
        (df_result.Group_OPO.isin(['Patent']))
        ],
       phasings_value,
       'TenderPackage'
       )

,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value
TenderPackage,,,,,,,,,,,,,,
BRANDED,0.00,0.00,"61,547,987,786.52","774,083,167,476.21","1,962,545,118,559.41","1,725,706,288,752.84","3,052,129,213,076.65",nan,nan,53.59,60.95,58.40,57.46,66.89
GENERIC 1,0.00,0.00,"48,567,420,561.60","433,384,295,437.26","1,258,988,302,259.03","1,154,055,525,050.60","1,383,021,860,216.44",nan,nan,42.29,34.13,37.46,38.43,30.31
GENERIC 2,0.00,0.00,"1,429,693,752.63","14,285,955,321.55","10,858,080,973.81","5,899,395,824.36","5,300,626,062.01",nan,nan,1.24,1.12,0.32,0.20,0.12
GENERIC 3,0.00,0.00,0.00,"17,563,636.36","80,136,363.64","552,798,343.26","201,326,175.60",nan,nan,0.00,0.00,0.00,0.02,0.00
GENERIC 4,0.00,0.00,"2,397,878,107.93","14,460,428,680.45","33,378,852,238.44","37,478,685,551.30","48,125,365,715.29",nan,nan,2.09,1.14,0.99,1.25,1.05
GENERIC 5,0.00,0.00,"913,238,964.34","33,710,057,277.73","94,712,815,106.91","79,409,672,665.71","74,386,404,320.55",nan,nan,0.80,2.65,2.82,2.64,1.63
All,0.00,0.00,"114,856,219,173.02","1,269,941,467,829.57","3,360,563,305,501.24","3,003,102,366,188.08","4,563,164,795,566.54",nan,nan,nan,nan,nan,nan,nan


1.4. OPO

In [14]:
report(df_result[
        (df_result.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])) &
        (df_result.Group_OPO.isin(['OPO']))
        ],
       phasings_value,
       'TenderPackage'
       )

,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value,2017_value,2018_value,2019_value,2020_value,2021_value,2022_value,2023_value
TenderPackage,,,,,,,,,,,,,,
BRANDED,0.00,"63,168,829,674.03","685,141,401,367.46","2,502,514,652,952.58","6,974,351,640,546.51","6,868,942,138,442.80","11,637,910,605,761.73",nan,45.65,32.27,28.98,32.18,30.79,37.51
GENERIC 1,0.00,"35,451,315,810.13","603,399,335,550.84","2,607,869,810,981.66","6,561,704,928,490.90","6,838,204,814,061.25","8,796,837,970,770.38",nan,25.62,28.42,30.20,30.28,30.65,28.35
GENERIC 2,0.00,"14,919,197,867.92","200,846,012,988.39","938,070,454,809.06","2,396,203,993,440.02","2,798,021,898,160.64","3,894,275,185,965.37",nan,10.78,9.46,10.86,11.06,12.54,12.55
GENERIC 3,0.00,"6,383,143,124.12","159,114,485,017.35","592,599,067,884.03","1,123,539,521,062.46","1,089,186,090,624.55","1,145,232,261,881.86",nan,4.61,7.49,6.86,5.18,4.88,3.69
GENERIC 4,0.00,"12,651,812,956.14","349,915,051,508.58","1,413,406,061,779.03","3,578,925,980,558.19","3,554,655,569,073.19","4,097,972,115,591.35",nan,9.14,16.48,16.37,16.51,15.93,13.21
GENERIC 5,0.00,"5,805,472,066.21","124,948,982,926.52","580,010,024,079.95","1,037,243,925,946.94","1,158,309,590,718.61","1,455,598,544,072.61",nan,4.20,5.88,6.72,4.79,5.19,4.69
All,0.00,"138,379,771,498.54","2,123,365,269,359.14","8,634,470,072,486.31","21,671,969,990,045.00","22,307,320,101,081.04","31,027,826,684,043.30",nan,nan,nan,nan,nan,nan,nan


**2. Volume (Inviting)**

2.1. All market

In [15]:
report(df_inviting[df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])],
       phasings_quota,
       'TenderPackage'
       )

,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume
TenderPackage,,,,,,,,,,,,,,
BRANDED,"178,512.67","11,842,184.60","59,304,763.50","173,969,502.51","389,168,735.92","288,032,689.11","493,671,320.97",9.58,4.83,2.07,2.14,2.75,2.33,3.01
GENERIC 1,"485,273.20","27,529,508.63","251,232,378.46","872,108,257.77","1,957,503,641.36","1,996,233,277.80","2,794,413,501.01",26.04,11.22,8.76,10.75,13.83,16.14,17.02
GENERIC 2,"332,808.31","30,949,151.66","310,270,051.62","1,019,568,275.64","2,038,822,892.78","2,068,801,707.18","3,255,331,921.52",17.86,12.61,10.82,12.57,14.40,16.73,19.82
GENERIC 3,"122,127.96","35,684,409.43","356,661,423.82","980,371,059.33","1,882,451,375.63","1,744,081,212.00","2,054,928,962.11",6.55,14.54,12.44,12.09,13.30,14.10,12.51
GENERIC 4,"435,909.10","129,856,483.77","1,789,406,882.21","4,782,840,740.99","7,514,963,463.06","6,062,738,025.13","7,592,922,619.71",23.39,52.93,62.43,58.96,53.10,49.02,46.24
GENERIC 5,"308,852.45","9,490,429.79","99,471,895.41","283,241,708.85","370,801,905.10","207,924,813.63","230,064,762.10",16.57,3.87,3.47,3.49,2.62,1.68,1.40
All,"1,863,483.68","245,352,167.87","2,866,347,395.02","8,112,099,545.09","14,153,712,013.84","12,367,811,724.84","16,421,333,087.42",nan,nan,nan,nan,nan,nan,nan


2.2. Patent & OPO

In [16]:
report(df_inviting[df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5']) &
       (df_inviting.Group_OPO.isin(['Patent','OPO']))],
       phasings_quota,
       'TenderPackage'
       )

,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume
TenderPackage,,,,,,,,,,,,,,
BRANDED,"178,512.67","11,511,019.29","52,106,318.30","130,796,031.66","327,348,087.74","244,343,394.95","428,624,691.72",11.45,9.71,5.14,5.02,6.37,4.89,5.83
GENERIC 1,"471,148.01","19,291,659.26","141,352,949.05","404,668,284.24","891,467,371.18","970,359,018.54","1,434,762,403.98",30.22,16.28,13.93,15.53,17.36,19.42,19.52
GENERIC 2,"315,437.58","21,367,014.63","129,414,389.29","392,647,012.08","845,594,735.22","1,027,290,909.10","1,820,485,224.98",20.23,18.03,12.76,15.06,16.46,20.56,24.76
GENERIC 3,"118,937.77","17,850,598.95","168,515,131.97","440,022,150.90","949,543,150.21","864,271,243.59","1,140,853,888.16",7.63,15.06,16.61,16.88,18.49,17.30,15.52
GENERIC 4,"223,109.24","42,670,603.77","484,753,204.80","1,148,628,350.46","2,031,761,016.02","1,854,755,639.44","2,482,657,938.34",14.31,36.00,47.78,44.07,39.56,37.13,33.77
GENERIC 5,"251,947.24","5,834,097.19","38,440,998.85","89,651,050.36","90,193,223.41","34,713,841.84","44,446,013.73",16.16,4.92,3.79,3.44,1.76,0.69,0.60
All,"1,559,092.52","118,524,993.10","1,014,582,992.26","2,606,412,879.69","5,135,907,583.77","4,995,734,047.47","7,351,830,160.92",nan,nan,nan,nan,nan,nan,nan


2.3. Patent

In [17]:
report(df_inviting[df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5']) &
       (df_inviting.Group_OPO.isin(['Patent']))],
       phasings_quota,
       'TenderPackage'
       )

,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume
TenderPackage,,,,,,,,,,,,,,
BRANDED,0.00,"35,248.55","1,407,885.84","7,878,002.42","22,516,779.30","15,689,954.94","20,780,757.97",nan,44.00,33.31,38.16,47.99,39.85,41.20
GENERIC 1,0.00,"10,284.56","875,131.07","5,359,704.40","14,375,136.79","12,983,394.20","16,669,250.06",nan,12.84,20.71,25.96,30.64,32.98,33.05
GENERIC 2,0.00,0.00,"30,245.54","198,192.62","362,543.81","2,331,724.74","2,672,950.34",nan,0.00,0.72,0.96,0.77,5.92,5.30
GENERIC 3,0.00,0.00,0.00,"40,651.82","350,102.07","487,929.96","508,661.68",nan,0.00,0.00,0.20,0.75,1.24,1.01
GENERIC 4,0.00,"34,585.04","1,800,974.32","6,604,053.20","8,216,423.06","7,138,011.13","8,482,581.08",nan,43.17,42.61,31.99,17.51,18.13,16.82
GENERIC 5,0.00,0.00,"112,368.42","563,227.50","1,101,729.18","737,928.49","1,319,389.20",nan,0.00,2.66,2.73,2.35,1.87,2.62
All,0.00,"80,118.15","4,226,605.19","20,643,831.96","46,922,714.23","39,368,943.45","50,433,590.33",nan,nan,nan,nan,nan,nan,nan


2.4. OPO

In [18]:
report(df_inviting[df_inviting.TenderPackage.isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5']) &
       (df_inviting.Group_OPO.isin(['OPO']))],
       phasings_quota,
       'TenderPackage'
       )

,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume,2017_volume,2018_volume,2019_volume,2020_volume,2021_volume,2022_volume,2023_volume
TenderPackage,,,,,,,,,,,,,,
BRANDED,"178,512.67","11,475,770.74","50,698,432.45","122,918,029.24","304,831,308.44","228,653,440.02","407,843,933.75",11.45,9.69,5.02,4.75,5.99,4.61,5.59
GENERIC 1,"471,148.01","19,281,374.70","140,477,817.99","399,308,579.83","877,092,234.38","957,375,624.34","1,418,093,153.92",30.22,16.28,13.90,15.44,17.24,19.32,19.42
GENERIC 2,"315,437.58","21,367,014.63","129,384,143.74","392,448,819.45","845,232,191.40","1,024,959,184.36","1,817,812,274.64",20.23,18.04,12.81,15.18,16.61,20.68,24.90
GENERIC 3,"118,937.77","17,850,598.95","168,515,131.97","439,981,499.08","949,193,048.13","863,783,313.64","1,140,345,226.49",7.63,15.07,16.68,17.02,18.65,17.43,15.62
GENERIC 4,"223,109.24","42,636,018.73","482,952,230.49","1,142,024,297.26","2,023,544,592.96","1,847,617,628.31","2,474,175,357.26",14.31,36.00,47.80,44.17,39.76,37.28,33.89
GENERIC 5,"251,947.24","5,834,097.19","38,328,630.43","89,087,822.86","89,091,494.23","33,975,913.35","43,126,624.53",16.16,4.93,3.79,3.45,1.75,0.69,0.59
All,"1,559,092.52","118,444,874.95","1,010,356,387.07","2,585,769,047.73","5,088,984,869.54","4,956,365,104.02","7,301,396,570.59",nan,nan,nan,nan,nan,nan,nan


**3. Winning Price**

In [ ]:
df_result['TenderYear'] = df_result['TenderYear'].astype(int)
df = df_result[(df_result['TenderPackage'].isin(['BRANDED','GENERIC 1','GENERIC 2','GENERIC 3','GENERIC 4','GENERIC 5'])) &
               df_result['TenderYear'].isin([2021,2022,2023]) &
               (df_result['Price'] > 0)]


pivot_winprice = pd.pivot_table(df,
                    values='QuotationID',
                    columns=['TenderPackage'],
                    index=['FormulaName'],
                    aggfunc=pd.Series.nunique,
                    margins=True
                    ).reset_index()

formula_list = pivot_winprice.sort_values(by=['All'],ascending = False).reset_index(drop=True)
formula_list = formula_list[~formula_list['FormulaName'].isin(['All',''])]

formula_list_incl_all = formula_list[(formula_list['BRANDED'].notna()) &
                            (formula_list['GENERIC 1'].notna()) &
                            (formula_list['GENERIC 2'].notna()) &
                            (formula_list['GENERIC 3'].notna()) &
                            (formula_list['GENERIC 4'].notna()) &
                            (formula_list['GENERIC 5'].notna())]

formula_list_add = formula_list[(~(formula_list['FormulaName'].isin(formula_list_incl_all['FormulaName']))) &
                            (formula_list['BRANDED'].notna()) &
                            (formula_list['GENERIC 1'].notna()) &
                            (formula_list['GENERIC 2'].notna()) &
                            (formula_list['GENERIC 3'].notna())
                            ][:100]

formula_list_100 = pd.concat([formula_list_incl_all,formula_list_add])

pivot_winprice = pd.pivot_table(df[(df['FormulaName'].isin(formula_list_100['FormulaName']))],
                    values='Price',
                    index=['FormulaName'],
                    columns=['TenderYear','TenderPackage'],
                    aggfunc='mean'
                    )

pivot_winprice.to_excel('pivot_winprice_processing.xlsx')